# 제외 컬럼
기존 3개 +     
person_prefer_d_3_attribute_d_m_contents_attribute_d_attribute_d_m  +  
person_prefer_h_3_attribute_h_l

###optuna 하이퍼 파라미터 최적화

#### 1.데이터 로드 및 전처리

In [ ]:
from google import colab
colab.drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install catboost
!pip install optuna

     |████████████████████████████████| 76.3 MB 1.3 MB/s 
     |████████████████████████████████| 308 kB 5.5 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 209 kB 46.2 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████████████████| 149 kB 51.6 MB/s 
     |████████████████████████████████| 112 kB 47.5 MB/s 
     |████████████████████████████████| 49 kB 6.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=eb1fd68a5cc35f1e787230ce3b5adf60549dfc029a40bbcfd6c3be29c5d4c92e
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import os
import sys
import platform
import random
import math
from typing import List ,Dict, Tuple

import pandas as pd
import numpy as np
 
import sklearn 
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score 
import optuna

from catboost import Pool,CatBoostClassifier

print(f"- os: {platform.platform()}")
print(f"- python: {sys.version}")
print(f"- pandas: {pd.__version__}")
print(f"- numpy: {np.__version__}")
print(f"- sklearn: {sklearn.__version__}")

- os: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
- python: 3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
- pandas: 1.1.5
- numpy: 1.19.5
- sklearn: 1.0.1


In [ ]:
#유진 ver2

train_data = pd.read_csv('/content/drive/MyDrive/Jobcare_data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Jobcare_data/test.csv')
SEED = 43
code_d = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_H_코드.csv')
code_l = pd.read_csv('/content/drive/MyDrive/Jobcare_data/속성_L_코드.csv')

train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Jobcare_data/'
train_data = pd.read_csv(f'{DATA_PATH}train.csv')
test_data = pd.read_csv(f'{DATA_PATH}test.csv')
SEED = 42
code_d = pd.read_csv(f'{DATA_PATH}속성_D_코드.csv').iloc[:,:-1]
code_h = pd.read_csv(f'{DATA_PATH}속성_H_코드.csv')
code_l = pd.read_csv(f'{DATA_PATH}속성_L_코드.csv')

train_data.shape , test_data.shape

((501951, 35), (46404, 34))

In [ ]:
code_d.columns= ["attribute_d","attribute_d_d","attribute_d_s","attribute_d_m"]
code_h.columns= ["attribute_h","attribute_h_m","attribute_h_l"]
code_l.columns= ["attribute_l","attribute_l_d","attribute_l_s","attribute_l_m","attribute_l_l"]

In [ ]:
code_l.columns.values[0]

'attribute_l'

#### 1.1 전처리 함수

In [ ]:
def merge_codes(df:pd.DataFrame,df_code:pd.DataFrame,col:str)->pd.DataFrame:#매개변수의 타입이 data.frame
#print(test.__annotations__) 참고로 import annotation해서 함수안 파라미터를 확인할 수 있다.
    df = df.copy()
    #df는 데이터 파일이 들어올 변수로 설정
    df_code = df_code.copy()
    #다른 속성코드 데이터 로드
    df_code = df_code.add_prefix(f"{col}_")
    #로드 데이터 마다 col명을 달고 인덱스를 시작하고자 ex) col_1  col_2 이런식
    df_code.columns.values[0] = col # col = attribute_l 이런 식으로 큰 이름만 따온다.
    return pd.merge(df,df_code,how="left",on=col) 
    # col기준으로 left조인, left는 df// 따라서 df기준으로 조인

def preprocess_data(
                    df:pd.DataFrame,
                    is_train:bool = True,
                    cols_merge:List[Tuple[str,pd.DataFrame]] = []  ,
                    cols_equi:List[Tuple[str,str]]= [] ,
                    cols_drop:List[str] = ["id","person_prefer_f","person_prefer_g" ,"contents_open_dt"]
                    )->Tuple[pd.DataFrame,np.ndarray]: #Tuple형태로 리턴하겠다.
    df = df.copy()

    y_data = None
    if is_train: # 처음에 value는 true이니 진행
        y_data = df["target"].to_numpy()
        #numpy값으로 target값을 y_data에 저장
        df = df.drop(columns="target")
        #target컬럼 제거

    for col, df_code in cols_merge:
        df = merge_codes(df,df_code,col)
        #left 조인함

    cols = df.select_dtypes(bool).columns.tolist()
    df[cols] = df[cols].astype(int)

    for col1, col2 in cols_equi:
        df[f"{col1}_{col2}"] = (df[col1] == df[col2] ).astype(int)

    df = df.drop(columns=cols_drop)
    return (df , y_data)
    #하나로 다 합쳐서 조인함

In [ ]:
#조인 키들 나열
# 소분류 중분류 대분류 속성코드 merge 컬럼명 및 데이터 프레임 리스트
cols_merge = [
              ("person_prefer_d_1" , code_d),
              ("person_prefer_d_2" , code_d),
              ("person_prefer_d_3" , code_d),
              ("contents_attribute_d" , code_d),
              ("person_prefer_h_1" , code_h),
              ("person_prefer_h_2" , code_h),
              ("person_prefer_h_3" , code_h),
              ("contents_attribute_h" , code_h),
              ("contents_attribute_l" , code_l),
]
# 회원 속성과 콘텐츠 속성의 동일한 코드 여부에 대한 컬럼명 리스트
cols_equi = [
    ("contents_attribute_c","person_prefer_c"),
    ("contents_attribute_e","person_prefer_e"),

    ("person_prefer_d_2_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_2_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_2_attribute_d_d" , "contents_attribute_d_attribute_d_d"),
    ("person_prefer_d_3_attribute_d_s" , "contents_attribute_d_attribute_d_s"),
    ("person_prefer_d_3_attribute_d_m" , "contents_attribute_d_attribute_d_m"),
    ("person_prefer_d_3_attribute_d_d" , "contents_attribute_d_attribute_d_d"),

    ("person_prefer_h_1_attribute_h_m" , "contents_attribute_h_attribute_h_m"),#attribute_h_m ,l,none
    ("person_prefer_h_1_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_2_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_3_attribute_h_m" , "contents_attribute_h_attribute_h_m"),
    ("person_prefer_h_2_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
    ("person_prefer_h_3_attribute_h_l" , "contents_attribute_h_attribute_h_l"),
]
# 학습에 필요없는 컬럼 리스트
cols_drop = ["id","person_prefer_f","person_prefer_g" ,
             "person_prefer_d_3_attribute_d_m_contents_attribute_d_attribute_d_m", "person_prefer_h_3_attribute_h_l"]

In [ ]:
x_train, y_train = preprocess_data(train_data, cols_merge = cols_merge , cols_equi= cols_equi , cols_drop = cols_drop)
#데이터 조인해서 train_data를 x와 y로 분리
x_test, _ = preprocess_data(test_data,is_train = False, cols_merge = cols_merge , cols_equi= cols_equi  , cols_drop = cols_drop)
#test에서는 타겟 값이 없으니 _로 표기
x_train.shape , y_train.shape , x_test.shape

((501951, 67), (501951,), (46404, 67))

In [ ]:
print(x_train.columns)

Index(['d_l_match_yn', 'd_m_match_yn', 'd_s_match_yn', 'h_l_match_yn',
       'h_m_match_yn', 'h_s_match_yn', 'person_attribute_a',
       'person_attribute_a_1', 'person_attribute_b', 'person_prefer_c',
       'person_prefer_d_1', 'person_prefer_d_2', 'person_prefer_d_3',
       'person_prefer_e', 'person_prefer_h_1', 'person_prefer_h_2',
       'person_prefer_h_3', 'contents_attribute_i', 'contents_attribute_a',
       'contents_attribute_j_1', 'contents_attribute_j',
       'contents_attribute_c', 'contents_attribute_k', 'contents_attribute_l',
       'contents_attribute_d', 'contents_attribute_m', 'contents_attribute_e',
       'contents_attribute_h', 'person_rn', 'contents_rn', 'contents_open_dt',
       'person_prefer_d_1_attribute_d_d', 'person_prefer_d_1_attribute_d_s',
       'person_prefer_d_1_attribute_d_m', 'person_prefer_d_2_attribute_d_d',
       'person_prefer_d_2_attribute_d_s', 'person_prefer_d_2_attribute_d_m',
       'person_prefer_d_3_attribute_d_d', 'person_prefer_

In [ ]:
cat_features = x_train.columns[x_train.nunique() > 2].tolist()
#날짜나 선호 특성에 대한 값이 낮은것(범주가 최대 2개인것)들을 제외하기 위함 

In [ ]:
is_holdout = False
#기존 train과 val데이터 분할 안함
n_splits = 5
#반복수
patience = 50
#최소 인내수
cv = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
#k-fold 5번 분할하고 교차 검증 진행하기 위함

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# create trial function
OPTUNA_OPTIMIZATION = True

def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(x_train,y_train, test_size=0.3)
    
    params = {
        'iterations':trial.suggest_int("iterations", 500, 3000),
        'objective':trial.suggest_categorical('objcetive',['CrossEntropy','Logloss']),
        'bootstrap_type':trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
        'od_wait':trial.suggest_int('od_wait', 500, 1000),
        #The number of iterations to continue the training early_stopping_rounds같은거 
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01,1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',20,50),
        'depth': trial.suggest_int('depth',1,15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,20),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'verbose': False,
        "eval_metric":'F1',
        "cat_features" : cat_features,
        "one_hot_max_size":trial.suggest_int("one_hot_max_size",1,5),
        'task_type' : 'GPU',
    }
    
    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    elif params['bootstrap_type'] == 'Bernoulli':
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1)
    
    model = CatBoostClassifier(**params)
    model.fit(
        train_x, train_y, eval_set=[(valid_x, valid_y)],
        #early_stopping_rounds=200,
        use_best_model=True
    )
    
    # validation prediction

    preds = model.predict(valid_x)
    pred_labels = np.rint(preds)
    score = f1_score(valid_y, pred_labels)
    return score

In [ ]:
study = optuna.create_study(
    direction='maximize',
    study_name='CatbClf'
)

study.optimize(
    objective, 
    n_trials=20
)

[I 2022-01-10 08:24:49,699] A new study created in memory with name: CatbClf
[I 2022-01-10 08:45:45,152] Trial 0 finished with value: 0.6383944475965535 and parameters: {'iterations': 2100, 'objcetive': 'Logloss', 'bootstrap_type': 'Bayesian', 'od_wait': 791, 'learning_rate': 0.9485611815975942, 'reg_lambda': 93.91856746837021, 'random_strength': 33.5708306171821, 'depth': 15, 'min_data_in_leaf': 14, 'leaf_estimation_iterations': 2, 'one_hot_max_size': 5, 'bagging_temperature': 4.136462998446275}. Best is trial 0 with value: 0.6383944475965535.
[I 2022-01-10 08:50:01,215] Trial 1 finished with value: 0.6642098126480992 and parameters: {'iterations': 993, 'objcetive': 'CrossEntropy', 'bootstrap_type': 'Bernoulli', 'od_wait': 969, 'learning_rate': 0.6682687151742356, 'reg_lambda': 30.615812543310096, 'random_strength': 41.779138993123055, 'depth': 6, 'min_data_in_leaf': 5, 'leaf_estimation_iterations': 9, 'one_hot_max_size': 4, 'subsample': 0.9591797714211849}. Best is trial 1 with value

In [ ]:
# 유진 ver.3 (person_prefer_d_3_attribute_d_m_contents_attribute_d_attribute_d_m, person_prefer_h_3_attribute_h_l 제거)

Best_params = study.best_trial.params
#dic2 = {"task_type":'GPU',"eval_metric":'F1',"cat_features":cat_features,"one_hot_max_size":4}
#Best_params.update(dic2)
print(f"Best Trial: {study.best_trial.value}")
print(f"Best Params: {study.best_trial.params}")

# Best Trial: 0.677971389388986
#Best Params: {'iterations': 1422, 'objcetive': 'CrossEntropy', 'bootstrap_type': 'Bayesian', 'od_wait': 666, 
'learning_rate': 0.9782109291187356, 'reg_lambda': 70.72533306533951, 'random_strength': 47.81900485462368, 'depth': 3, 
'min_data_in_leaf': 20, 'leaf_estimation_iterations': 5, 'one_hot_max_size': 1, 'bagging_temperature': 0.07799233624102353}

Best Trial: 0.677971389388986
Best Params: {'iterations': 1422, 'objcetive': 'CrossEntropy', 'bootstrap_type': 'Bayesian', 'od_wait': 666, 'learning_rate': 0.9782109291187356, 'reg_lambda': 70.72533306533951, 'random_strength': 47.81900485462368, 'depth': 3, 'min_data_in_leaf': 20, 'leaf_estimation_iterations': 5, 'one_hot_max_size': 1, 'bagging_temperature': 0.07799233624102353}
